## Julia Codes of Chap3

In [29]:
using LinearAlgebra

In [30]:
function forwardrow(L::Array{Float64,2}, b::Array{Float64,1})
    # FORWARDROW forward substitution: row oriented version.
    # X=FORWARDROW(L,B) solve the lower triangular system L*X=B with
    # the forward substitution method in the row-oriented version.
    local x::Array{Float64}
    
    n, m = size(L)
    if n != m
        println("Only squared systems")
    end
    if minimum(abs.(diag(L))) == 0
        println("The system is singular")
    end
    
    # x = zeros(n) useless
    x = zeros(Float64, n)
    x[1] = b[1] / L[1, 1]
    for i = 2:n
        a = L[i, 1:i-1] .* x[1:i-1] # Float64/array
        x[i] = (b[i] - a[1, 1]) / L[i, i]
    end
    return x
end

forwardrow (generic function with 1 method)

In [31]:
function forwardcol(L, b)
    # FORWARDCOL forward substitution: column oriented version.
    # X=FORWARDCOL(L,B) solve the lower triangular system L*X=B with the
    # forward substitution method in the column-oriented version.
    n, m = size(L)
    if n != m
        println("Only squared systems")
    end
    if minimum(abs.(diag(L))) == 0
        println("The system is singular")
    end
    for j = 1:n-1
        b[j] = b[j] / L[j,j]
        b[j+1:n] = b[j+1:n] - b[j] * L[j+1:n,j] 
    end 
    b[n] = b[n] / L[n, n]
    return b
end

forwardcol (generic function with 1 method)

In [32]:
function backwardcol(U::Array{Float64,2}, b)
    # BACKWARDCOL backward substitution: column oriented version.
    #  X=BACKWARDCOL(U,B) solve the upper triangular system U*X=B with the
    #  backward substitution method in the column-oriented version.
    local x::Array{Float64}
    
    n, m = size(U)
    if n != m
        println("Only squared systems")
    end
    if minimum(abs.(diag(U))) == 0
        println("The system is singular")
    end
    for j = n:-1:2
        b[j] = b[j] / U[j, j]
        b[1:j-1] = b[1:j-1] - b[j] * U[1:j-1, j] 
    end
    b[1] = b[1] / U[1,1]
    return b
end  

backwardcol (generic function with 1 method)

In [33]:
N = 20000
L = rand(N, N)
b = rand(N)
if minimum(diag(L)) < 0
    println("false")
end

In [34]:
@time x1 = forwardrow(L, b)
@time x2 = forwardcol(L, b);
@time x3 = backwardcol(L, b);

@time x1 = forwardrow(L, b)
@time x2 = forwardcol(L, b);
@time x3 = backwardcol(L, b);

  4.213641 seconds (180.95 k allocations: 4.480 GiB, 6.16% gc time)
  0.921752 seconds (205.89 k allocations: 5.971 GiB, 14.00% gc time)
  0.926341 seconds (215.87 k allocations: 5.972 GiB, 14.09% gc time)
  4.434708 seconds (113.86 k allocations: 4.477 GiB, 2.64% gc time)
  1.123692 seconds (151.81 k allocations: 5.969 GiB, 13.74% gc time)
  1.096752 seconds (151.81 k allocations: 5.969 GiB, 14.18% gc time)


### Modified Gram-Schmidt Method

In [35]:
function modgrams(A::Array{Float64,2})
    # MODGRAMS QR factorization of a matrix A.
    # [Q,R]=MODGRAMS(A) produces an upper trapezoidal matrix R and an orthogonal 
    # matrix Q such that Q*R=A.
    m, n = size(A)
    Q = zeros(Float64, m, n)   
    Q[1:m, 1] = A[1:m, 1]  
    R = zeros(Float64, n, n)  
    R[1, 1] = 1
    for k = 1:n
        R[k, k] = norm(A[1:m, k])
        Q[1:m, k] = A[1:m, k] / R[k, k]
        j = k+1:n  # UnitRange or Array, different from Matlab
        # println(Q)
        # println(A)
        a = Q[1:m, k]' * A[1:m, j]
        R[k, j] = a'
        b = kron(Q[1:m, k]', R[k, j])
        # println(Q[1:m,k]', R[k,j])
        # println(b)
        A[1:m, j] = A[1:m, j] - kron(Q[1:m, k]', R[k, j])'  # tensor product different from Matlab
    end
    return Q, R
end

modgrams (generic function with 1 method)

In [36]:
A = [1.0 2.0 3.0; 4.0 5.0 6.0; 4.0 3.0 3.0];
Q, R = modgrams(A)

([0.17407765595569785 0.5627039587626639 0.8081220356417684; 0.6963106238227914 0.5099504626286637 -0.5050762722761051; 0.6963106238227914 -0.6506264523193306 0.3030457633656641], [5.744562646538029 5.918640302493727 6.789028582272217; 0.0 1.7232808737106582 2.7959352951019865; 0.0 0.0 0.3030457633656631])

### LU factorization

In [58]:
function lukji(A::Array{Float64,2})
    # LUKJI LU factorization of a matrix A in the kji version
    # Y=LUKJI(A) Y contains the strict lower triangle of L embedded in the same 
    # matrix as the upper triangle of U.
    n, m = size(A)
    if n != m
        println("Only squared systems")
    end
    for k = 1:n-1
        if A[k, k] == 0
            println("Null pivot element")
        end
        a = A[k+1:n, k] / A[k, k]
        A[k+1:n, k] = a'
        for j = k+1:n
        i = k+1:n
        A[i, j] = A[i, j] - A[i, k] * A[k, j]
        end
    end
    return A
end

lukji (generic function with 1 method)

In [59]:
function lukij(A::Array{Float64,2})
    # LUKJI LU factorization of a matrix A in the kji version
    # Y=LUKJI(A) Y contains the strict lower triangle of L embedded in the same 
    # matrix as the upper triangle of U.
    n, m = size(A)
    if n != m
        println("Only squared systems")
    end
    for k = 1:n-1
        if A[k, k] == 0
            println("Null pivot element")
        end
        a = A[k+1:n, k] / A[k, k]
        A[k+1:n, k] = a'
        for i = k+1:n
        j = k+1:n
        A[i, j] = A[i, j] - A[i, k] * A[k, j]
        end
    end
    return A
end

lukij (generic function with 1 method)

In [60]:
function lujki(A::Array{Float64,2})
    # LUJKI LU factorization of a matrix A in the jki version
    # Y=LUJKI(A) Y contains the strict lower triangle of L embedded in the same 
    # matrix as the upper triangle of U.
    n, m = size(A)
    if n != m
        println("Only squared systems") 
    end
    for j = 1:n
        if A[j, j] == 0
            println("Null pivot element") 
        end
        for k = 1:j-1   
            i = k+1:n 
            A[i, j] = A[i, j] - A[i, k] * A[k, j]
        end
        i = j+1:n   
        A[i, j] = A[i, j] / A[j, j] 
    end
    return A
end

lujki (generic function with 1 method)

In [61]:
A = [-149.0   -50.0  -154.0; 537.0   180.0   546.0; -27.0    -9.0   -25.0];
x1 = lujki(A)
x2 = lukij(A)
x3 = lukji(A)

3×3 Array{Float64,2}:
 -149.0          -50.0       -154.0    
   -0.000162336    0.999946    -5.32017
    8.16216e-6    -0.357533    -3.78343

In [63]:
N = 2000
A = rand(N, N)
b = rand(N)
if minimum(diag(L)) < 0
    println("false")
end

@time lukji(A)
@time lukij(A)
@time lujki(A)


 17.819554 seconds (8.00 M allocations: 80.604 GiB, 11.47% gc time)
 45.499564 seconds (8.00 M allocations: 80.604 GiB, 5.92% gc time)
 17.295712 seconds (8.00 M allocations: 80.604 GiB, 13.33% gc time)


2000×2000 Array{Float64,2}:
 0.582883     0.0903721    0.450906   …      0.909356      0.321684
 2.75837     -0.352957    -2.19844          -3.85586      -0.844279
 4.30789    100.185      184.393           331.962        82.0589  
 2.48225    240.967        2.87934         -25.9085      -25.7804  
 4.0829     429.327        5.13101       -1144.05      -2028.36    
 1.25628    231.819        2.77264    …   -271.079      -315.579   
 3.05837    308.719        3.68922         322.36        221.493   
 4.72359    534.734        6.39179        9576.47       6461.01    
 4.25879      2.35878      0.0179518       -36.6838      -29.3772  
 0.0683152  417.154        4.99476          13.4421        9.31137 
 0.922119   392.69         4.69972    …     52.8155       39.4901  
 1.84496    156.141        1.86509           0.086853     -0.682989
 3.74931     75.4725       0.894554         -0.388761     -0.639987
 ⋮                                    ⋱                            
 3.22885     15.5813

### Cholesky factorization

In [41]:
function choles2(A)
    # CHOL2 Cholesky factorization of a s.p.d. matrix A.
    # R=CHOL2(A) produces an upper triangular matrix R such that
    # R'*R=A.
    n, m = size(A)
    if n != m
        println("Only squared systems")
    end
    for k = 1:n-1
        if A[k, k] <= 0
            println("Null or negative pivot element")
        end
        A[k, k] = sqrt(A[k, k])
        A[k+1:n, k] = A[k+1:n, k] / A[k, k]
        for j = k+1:n
            A[j:n, j] = A[j:n, j] - A[j:n, k] * A[j, k]
        end
    end
    A[n, n] = sqrt(A[n, n])
    A = tril(A) 
    A = A'
    return A
end

choles2 (generic function with 1 method)

In [42]:
A = [2.0   1.0  0.0; 1.0   2.0   1.0; 0.0    1.0   2.0];
R = choles2(A)

3×3 Adjoint{Float64,Array{Float64,2}}:
 1.41421  0.707107  0.0     
 0.0      1.22474   0.816497
 0.0      0.0       1.1547  

### Banded matrix

In [43]:
function luband(A, p, q)
    #  LUBAND LU factorization for a banded matrix
    #  Y=LUBAND(A,P,Q) Y contains the strict lower triangle of L embedded in 
    #  the same matrix as the upper triangle of U for a banded matrix A
    #  with an upper bandwidth Q and a lower bandwidth P.
    n, m = size(A)
    if n != m 
        println("Only squared systems")
    end
    for k = 1:n-1
        for i = k+1:min(k+p, n)
            A[i, k] = A[i, k] / A[k, k]
        end
        for j = k+1:min(k+q, n)
            i = k+1:min(k+p, n)   
            A[i, j] = A[i, j] - A[i, k] * A[k, j]
        end
    end
    return A
end

luband (generic function with 1 method)

In [44]:
A = [2.0   1.0  0.0  0.0;  1.0   2.0   1.0  0.0;  0.0  1.0   2.0  1.0; 0.0 0.0 1.0 2.0];
Y = luband(A, 1, 1)

4×4 Array{Float64,2}:
 2.0  1.0       0.0      0.0 
 0.5  1.5       1.0      0.0 
 0.0  0.666667  1.33333  1.0 
 0.0  0.0       0.75     1.25

In [45]:
function forwband(L, p, b)
    #  FORWBAND forward substitution for a banded matrix
    #  X=FORWBAND(L,P,B) solve the lower triangular system L*X=B
    #  where L is a matrix with lower bandwidth P.
    n, m = size(L)
    if n != m
        print("Only squared systems")
    end
    for j = 1:n-1
        i = j+1:minimum([j+p, n]) 
        a = i[1]
        b[a] = b[a] - L[a, j] * b[j]  
    end
    return b
end

L = [1.0000         0         0         0;
     0.5000    1.0000         0         0;
          0    0.6667    1.0000         0;
          0         0    0.7500    1.0000];
b = [0; 1.0; 1.0; 0];
x = forwband(L, 1, b)

4-element Array{Float64,1}:
  0.0                
  1.0                
  0.33330000000000004
 -0.24997500000000003

In [46]:
function backband(U, q, b)
    #  BACKBAND forward substitution for a banded matrix
    #  X=BACKBAND(U,Q,B) solves the upper triangular system U*X=B
    #  where U is a matrix with upper bandwidth Q.
    n, m = size(U)
    if n != m
        println("Only square systems")
    end
    for j = n:-1:1
        b[j] = b[j] / U[j, j]
        i = max(1,j-q):j-1
        b[i] = b[i] - U[i, j] * b[j] 
    end
    return b
end

U = [2.0000    1.0000         0         0
         0    1.5000    1.0000         0
         0         0    1.3333    1.0000
         0         0         0    1.2500];
b = [0; 1.0; 1.0; 0];
x = backband(U, 1, b)

4-element Array{Float64,1}:
 -0.08332708317707942
  0.16665416635415883
  0.7500187504687618 
  0.0                

### Tomas

In [47]:
function modthomas(a, b, c, f)
    #  MODTHOMAS modified version of the Thomas algorithm
    #  X=MODTHOMAS(A,B,C,F) solve the system T*X=F where T
    #  is the tridiagonal matrix T=tridiag(B,A,C).
    
    gamma = zeros(4)  # need to be defined
    y = zeros(4)      # data deliver can not write y = gamma
    x = zeros(4)
    
    n = length(a)
    b = [0; b] 
    c = [c; 0] 
    gamma[1] = 1 / a[1]
    for i = 2:n
        gamma[i] = 1 / (a[i] - b[i] * gamma[i-1] * c[i-1])
    end
    y[1] = gamma[1] * f[1]
    for i = 2:n
        y[i] = gamma[i] * (f[i] - b[i] * y[i-1])
    end
    x[n,1] = y[n]
    for i = n-1:-1:1  
        x[i,1] = y[i] - gamma[i] * c[i] * x[i+1, 1]
    end
    return x
end

modthomas (generic function with 1 method)

In [48]:
b = ones(3, 1); c = ones(3, 1); a = 2*ones(4, 1);
f = [0; 1; 1; 0];
x = modthomas(a, b, c, f)

4-element Array{Float64,1}:
 -0.19999999999999998
  0.39999999999999997
  0.4                
 -0.2                

### Condition number

In [49]:
function condest2(A, L, U, theta)
    #  CONDEST2 Condition number
    #  K1=CONDEST2(A,L,U,THETA) returns an approximation of the
    #  condition number of a matrix A. L and U are the factor of
    #  the LU factorization of A. THETA contains random numbers.
    
    z = zeros(4)
    
    n, m = size(A)
    if n != m 
        println("Only squared systems")
    end
    p = zeros(1, n)
    for k = 1:n
        zplus = (theta[k] - p[k]) / U[k, k]  
        zminu = (-theta[k] - p[k]) / U[k, k]
        splus = abs(theta[k] - p[k])
        sminu = abs(-theta[k] - p[k])
        for i = k+1:n
            splus = splus + abs(p[i] + U[k, i] * zplus) 
            sminu = sminu + abs(p[i] + U[k, i] * zminu)  
        end   
        if splus >= sminu
            z[k] = zplus
        else
            z[k] = zminu
        end
        i = k+1:n
        p[i] = p[i] + U[k, i] * z[k]
    end
    z = z'
    x = backwardcol(L, z)
    w = forwardcol(L, x)  
    y = backwardcol(U, w)
    k1 = norm(A, 1) * norm(y, 1) / norm(x, 1)
    return k1
end

condest2 (generic function with 1 method)

In [50]:
A = [2     1     0     0
     1     2     1     0
     0     1     2     1
     0     0     1     2];
L = [1.0000         0         0         0
     0.5000    1.0000         0         0
          0    0.6667    1.0000         0
          0         0    0.7500    1.0000];
U = [2.0000    1.0000         0         0
          0    1.5000    1.0000         0
          0         0    1.3333    1.0000
          0         0         0    1.2500];
n, m = size(A)
theta = 0.1 * rand(n, 1);

In [51]:
k = condest2(A, L, U, theta)

14.0

### Cuthill-McKee Algorithm

In [52]:
using SparseArrays, CuthillMcKee, UnicodePlots
N = 500_000
A = sprand(N, N, 1/N)
A = A+A'
@time p = symrcm(A)
AP = rcmpermute(A)
display(spy(A))
display(spy(AP))

                       Sparsity Pattern
          ┌──────────────────────────────────────────┐    
        1 │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ > 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ < 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿

                       Sparsity Pattern
          ┌──────────────────────────────────────────┐    
        1 │⠻⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
          │⠀⠀⠻⣦⣄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
          │⠀⠀⠀⢹⣿⣿⣿⣶⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⢻⣿⣿⣿⣿⣿⣷⣦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠻⣿⣿⣿⣿⣿⣿⣿⣷⣦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠹⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠘⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣿⣿⣿⣿⣿⣿⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀

  0.181423 seconds (1.00 M allocations: 80.585 MiB, 9.31% gc time)
